In [1]:
from datasets import load_dataset

In [2]:
bookcorpus = load_dataset("bookcorpus", split="train")

In [3]:
bookcorpus

Dataset({
    features: ['text'],
    num_rows: 74004228
})

In [4]:
print(bookcorpus[0])

{'text': 'usually , he would be tearing around the living room , playing with his toys .'}


In [12]:
text_lengths = bookcorpus.map(lambda x: {"text_length": len(x["text"])}, num_proc=16, remove_columns=["text"])
total_length = sum(text_lengths["text_length"])
mean = total_length / len(bookcorpus)
mean

Map (num_proc=16):   0%|          | 0/74004228 [00:00<?, ? examples/s]

61.338552845926586

In [13]:
print(text_lengths[:10])

{'text_length': [78, 62, 62, 198, 121, 36, 10, 31, 61, 104]}


In [14]:
mrpc = load_dataset("glue", "mrpc")
mrpc

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [7]:
import os
import sys
print(os.getcwd())
print(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.getcwd()))
# print(sys.path)
from src.utils import translate_to_phonetic


/home/toure215/BERT_phonetic/test
/home/toure215/BERT_phonetic


In [35]:
mrpc = mrpc.remove_columns(["sentence2", "label", "idx"]).rename_column("sentence1", "text")
mrpc


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 408
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1725
    })
})